# DataFrame Practice
Here we will use spark dataframe on actual datasets.

## 1. DataFrames with JSON files

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local[*]').appName('spark_df_data').getOrCreate()

We can use the `json` reader to read in many json files at once.  Each json file becomes a single row in the resulting DataFrame:

In [ ]:
df = spark.read.json('structured-2018-01-14-neworleans/*.json')

In [ ]:
# how many rows do we have (i.e. how many json files did we read?)
df.count()

In [ ]:
# use the .columns member to list the columns out
df.columns

In [ ]:
# we can select columns just like in SQL
df.select(['map', 'mode', 'title']).show(5)

In [ ]:
# selecting a single column is similar to pandas
df.mode

In [ ]:
df['mode']

We can perform standard aggregations (e.g. avg, min, max, etc).

However, we always need to perform a `groupBy()`, even if we aren't grouping:

In [ ]:
df.groupBy().avg('duration_ms').show()

There are actually quite a few alternative syntaxes to do the same thing.  Sometimes this gets a little confusing:

In [ ]:
df.groupBy().agg({'duration_ms': 'avg'}).show()

In [ ]:
from pyspark.sql import functions as fn

#select avg(column) as avg_column
#avg(column) avg_column

df.groupBy().agg(fn.avg('duration_ms')).show()
df.groupBy().agg(fn.avg(fn.col('duration_ms'))).show()
df.groupBy().agg(fn.avg(fn.col('duration_ms').alias('duration'))).show()

In [ ]:
df.groupBy().agg(fn.avg(df.duration_ms)).show()

There are many useful functions in the `pyspark.sql.functions` module.  We will use some of them, like we did above.

Obviously, we can also perform aggregations over actual groups.  Here's an example:

In [ ]:
df.groupBy('mode').avg('duration_ms').show()

Json data is usually nested, which is a little "weird" when you are trying to analyze it using SQL-like tables.

For example, in the CWL json the `teams` field is actually a list of length 2 (one for each team):

In [ ]:
df.select('teams').limit(5).show()

Sometimes it can be better to do a `take` than a `show` so that we can see the nested structure better:

In [ ]:
df.select('teams').take(1)

The .explode() function is a very useful way to "denormalize" the data.  TL;DR explodes a nested list into multiple rows (at the cost of introducing some redundancy):

In [ ]:
from pyspark.sql import functions as fn
teams_df = df.select('id', fn.explode('teams'))
teams_df.show(5)

Actually, it is better to rename our column to "team" because we exploded a list of 2 teams into a 2 separate rows each containing a team.  We use `alias` to rename:

In [ ]:
teams_df = df.select('id', fn.explode('teams').alias('team'))
teams_df.show(5)

In [ ]:
teams_df.take(1)

Notice that the "team" column is still nested.  This isn't really limiting, though.  We can use the col.field syntax to get at the subfields:

In [ ]:
teams_df.select('id', 'team.name').show(5)

If we want to rename the column then we have to use a noisier syntax:

In [ ]:
teams_df.select('id', teams_df.team['name'].alias('team_name')).show(5)

... or we could've used the `.withColumnRenamed()` method:

In [ ]:
teams_df.select('id', 'team.name').withColumnRenamed('name', 'team_name').show(5)

Let's explode the `players` nested field:

In [ ]:
players_df = df.select('id', fn.explode('players'))
players_df.show(5)

To reduce the number of joins we'll have to make, let's redo this last step but keep some more fields (at the cost of redundancy).  This is "denormalization":

In [ ]:
players_df = df.select('id',
                       'platform',
                       'title',
                       'mode',
                       'map',
                       'start_time_s',
                       'end_time_s',                                                                                                                                                                                                                                                                                                                                                                                              
                       'duration_ms',
                       fn.explode('players').alias('player'))
players_df.show(5)

In [ ]:
players_df.take(1)

You can join just like in SQL

In [ ]:
joined_df = players_df.join(teams_df, 
                            [players_df.id == teams_df.id, 
                             players_df.player['team'] == teams_df.team['name']])
joined_df.show(5)

In [ ]:
#joined_df = players_df.join(teams_df, 
#                            ['id'])
#joined_df.show(5)

Filtering is also easy

In [ ]:
joined_df.select('mode').distinct().collect()

In [ ]:
ctf_df = joined_df.filter(joined_df.mode == 'Search & Destroy')

In [ ]:
ctf_df.count()

In [ ]:
joined_df.groupBy('mode').count().show()

In [ ]:
df.groupBy('mode').count().show()
#each game has eight players in total

## 2. Prepare and understand data for modeling

### Duplicates

Consider the following example.

In [ ]:
df_olympia = spark.read.options(header=True, inferSchema=True).csv('./olympia2016_athletes_ex.csv')
df_olympia.show(50)

In [ ]:
df_olympia.printSchema()

Check for duplicates.

In [ ]:
print(df_olympia.count())
print(df_olympia.distinct().count())

If these two numbers differ - you have rows that are exact copies of each other. We can drop these rows by using the `.dropDuplicates(...)` method.

In [ ]:
df_olympia = df_olympia.dropDuplicates()
df_olympia.show(20)

Let's confirm.

In [ ]:
print(df_olympia.count())
print(df_olympia.distinct().count())

We still have one more duplicate. We will use the `.dropDuplicates(...)` but add the `subset` parameter.

In [ ]:
#drop duplicates based on a subset of columns
#df_olympia = df_olympia.dropDuplicates(subset=df.columns[1:])
#df_olympia.show()

To calculate the total and distinct number of IDs in one step we can use the `.agg(...)` method.

In [ ]:
import pyspark.sql.functions as fn

df_olympia.agg(
    fn.count('id').alias('count'),
    fn.countDistinct('id').alias('distinct')
).show()

Give each row a unique ID. 

In [ ]:
df_olympia.withColumn('new_id', fn.monotonically_increasing_id()).show()

### Missing observations

Consider a similar example to the one we presented above.

To find the number of missing observations per row we can use the following snippet.

In [ ]:
df_olympia.rdd.repartition(4).map(
    lambda row: (row['id'], sum([c == None for c in row]))
).take(20)

Let's see what values are missing so when we count missing observations in columns we can decide whether to drop the observation altogether or impute some of the observations.

In [ ]:
df_olympia.where('id == 222063859').show()

What is the percentage of missing observations we see in each column?

In [ ]:
df_olympia.agg(*[
    (1 - (fn.count(c) / fn.count('*'))).alias(c + '_missing')
    for c in df_olympia.columns
]).show()

We will drop the `'income'` feature as most of its values are missing.

To drop the observations instead you can use the `.dropna(...)` method.

In [ ]:
df_olympia.dropna().show()

In [ ]:
df_olympia.dropna().count()

In [ ]:
df_olympia.fillna(0, ['gold', 'silver', 'bronze']).where('id = 256673338').show()

To impute a mean, median or other *calculated* value you need to first calculate the value, create a dict with such values, and then pass it to the `.fillna(...)` method.

In [ ]:
df_olympia = df_olympia.fillna(0, ['gold', 'silver', 'bronze'])
df_olympia.show()

In [ ]:
means = df_olympia.agg(fn.mean('height').alias('height')).toPandas().to_dict('records')[0]
df_olympia.fillna(means, 'height').show()

In [ ]:
df_olympia.agg(fn.mean('height').alias('height')).toPandas().head()

In [ ]:
from pyspark.sql import Window

window = Window().partitionBy('sport', 'sex')
df_olympia.withColumn('height', fn.when(fn.col('height').isNull(), fn.avg(fn.col('height')).over(window)).otherwise(fn.col('height'))).where('id = 222063859').show()

In [ ]:
window = Window().partitionBy('sport', 'sex')
df_olympia = df_olympia.withColumn('height', fn.when(fn.col('height').isNull(), fn.avg(fn.col('height')).over(window)).otherwise(fn.col('height')))
df_olympia.where('id = 222063859').show()

In [ ]:
df_olympia = df_olympia.withColumn('weight', fn.when(fn.col('weight').isNull(), fn.avg(fn.col('weight')).over(window)).otherwise(fn.col('weight')))
df_olympia.where('id = 587168078').show()

### Outliers

Consider another simple example.

In [ ]:
df_olympia.show()

First, we calculate the lower and upper *cut off* points for each feature.

In [ ]:
cols = ['height', 'weight']
bounds = {}

for col in cols:
    quantiles = df_olympia.approxQuantile(col, [0.25, 0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR]

The `bounds` dictionary holds the lower and upper bounds for each feature. 

In [ ]:
bounds

Let's now use it to flag our outliers.

In [ ]:
df_outliers = df_olympia.select(['id'] + [
    (
        (df_olympia[c] < bounds[c][0]) | 
        (df_olympia[c] > bounds[c][1])
    ).alias(c + '_o') for c in cols
])
df_outliers.show()

We have two outliers in the `weight` feature and two in the `age` feature.

In [ ]:
df_olympia_outliers = df_olympia.join(df_outliers, 'id')
df_olympia_outliers.filter('height_o').show()
df_olympia_outliers.filter('weight_o').show()

In [ ]:
#save the cleaned data
df_olympia.write.mode('overwrite').option('header', 'True').csv('./olympia2016_athletes_cleaned.csv')

## 3. Understand your data

### Descriptive statistics

Load our data and convert it to a Spark DataFrame.

In [ ]:
df_olympia = spark.read.options(header=True, inferSchema=True).csv('./olympia2016_athletes_cleaned.csv')

Next, we read the data in.

Following, we create the schema for our `DataFrame`.

Finally, we create our `DataFrame`.

In [ ]:
df_olympia.show()

Now that the dataframe is ready we can calculate the basic descriptive statistics for our dataset. 

In [ ]:
df_olympia.printSchema()

For categorical columns we will count the frequencies of their values using `.groupby(...)` method.

In [ ]:
df_olympia.groupBy('sex').count().show()

For the truly numerical features we can use the `.describe()` method.

In [ ]:
medals = ['gold', 'silver', 'bronze']

In [ ]:
df_olympia.describe(medals).show()

Here's how you check skewness (we will do it for the `'balance'` feature only).

In [ ]:
df_olympia.agg({'gold': 'skewness'}).show()

Which player won the most gold medals?

In [ ]:
win = Window.partitionBy()
df_olympia.withColumn("max_gold", fn.max("gold").over(win)).filter("max_gold = gold").drop("max_gold").show()

In [ ]:
win = Window.partitionBy('nationality')
df_olympia.withColumn("max_gold", fn.max("gold").over(win)).filter("max_gold = gold").drop("max_gold").show()

In [ ]:
df_olympia.groupBy('nationality').count().show()
df_olympia.groupBy('nationality').agg(fn.count('nationality').alias('count_player')).show()

In [ ]:
df_olympia.groupBy('nationality').avg('height').show()
df_olympia.groupBy('nationality').agg(fn.avg('height').alias('avg_height')).show()

In [ ]:
df_olympia.groupBy('nationality').sum('gold').show()

In [ ]:
df_olympia.createOrReplaceTempView('df_olympia')

In [ ]:
spark.sql('select * from df_olympia where gold = (select max(gold) from df_olympia)').show()
spark.sql('select * from df_olympia a, (select nationality, max(gold) as max_gold from df_olympia group by nationality) b where a.nationality=b.nationality and a.gold = b.max_gold').show()

In [ ]:
spark.sql('select nationality, count(*) from df_olympia group by nationality').show()

In [ ]:
spark.sql('select nationality, avg(height) from df_olympia group by nationality').show()
spark.sql('select nationality, avg(height) as avg_height from df_olympia group by nationality').show()

In [ ]:
spark.sql('select nationality, sum(gold) from df_olympia group by nationality').show()

### Correlations

Calculating correlations in PySpark is very easy once your data is in a DataFrame form.

In [ ]:
df_olympia.corr('height', 'gold')

In order to create a correlations matrix you can use the script below.

In [ ]:
hw_medals = ['height', 'weight', 'gold', 'silver', 'bronze']
len_hw_medals = len(hw_medals)

corr = []

for i in range(0, len_hw_medals):
    
    temp = [None] * i
    
    for j in range(i, len_hw_medals):
        temp.append(df_olympia.corr(hw_medals[i], hw_medals[j]))
    corr.append(temp)
    
corr

### Visualization

First, let's load the modules and set them up.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

### Histograms

Aggreagate the data in workers and return aggregated list of cut-off points and counts in each bin of the histogram to the driver.

In [ ]:
hists = df_olympia.select('gold').rdd.flatMap(lambda row: row).histogram(20)

In [ ]:
hists

To plot the histogram you can simply call the matplotlib like below.

In [ ]:
data = {
    'bins': hists[0][:-1],
    'freq': hists[1]
}

fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(1, 1, 1)
ax.bar(data['bins'], data['freq'])
ax.set_title('Histogram of gold medals')

#plt.savefig('gold_hist.png', dpi=300)

In a similar manner, a histogram can be create with Bokeh.

If your data is small enough to fit on the driver (although we would argue it would normally be faster to use the method showed above) you can bring the data and use the `.hist(...)` (from Matplotlib) or `.Histogram(...)` (from Bokeh) methods.

In [ ]:
data_to_driver = df_olympia.select('gold').rdd.flatMap(lambda row: row).collect()

In [ ]:
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(1, 1, 1)

ax.hist(data_to_driver, bins=20)
ax.set_title('Histogram of gold medals using .hist()')

#plt.savefig('Hist_Gold_2.png', dpi=300)

### Interactions between features

In this example we will sample our fraud dataset at 1% given gender as strata.

In [ ]:
plot_col = ['height', 'gold']
data_sample = df_olympia.sampleBy('sex', {'female': 0.1, 'male': 0.1}).select(plot_col)
data_multi = dict([
    (elem, data_sample.select(elem).rdd.flatMap(lambda row: row).collect()) 
    for elem in plot_col
])

To put multiple 2D charts in one go you can use

In [ ]:
fig = plt.figure(figsize=(12,9))
ax = fig.add_subplot(1, 1, 1)
ax.scatter(data_multi['height'], data_multi['gold'])
ax.set_title('Scatter plot of height and gold medals')